In [27]:
import numpy as np
def get_condition_indices(stabilizer_shape):
    """
    Extracts the list of relative positions (dx, dy) from a m x n binary rule matrix
    that define which neighbor cells contribute to the update rule.

    The (0, *) row is ignored (assumed to represent the current cell),
    and only rows over 1 are used to define condition offsets.
    """
    rows, cols = stabilizer_shape.shape
    print(f"Stabilizer shape:\n{stabilizer_shape}")
    print(f"Rows: {rows}, Cols: {cols}")
    # set (0, 0) for all row 0
    
    relative_positions = [[(i, j) for j in range(-cols//2 + 1, cols//2 + 1)] for i in range(rows)]
    for j in range(cols):
        relative_positions[0][j] = (0, 0)
        
    print(f"Relative positions:\n{relative_positions}")

    active_conditions = []               

    for i in range(1, rows):
        for j in range(cols):
            if stabilizer_shape[i][j] == 1:
                active_conditions.append(relative_positions[i][j])

    return active_conditions

def fill_Z_with_stabilizer_shape(input_row, H, L, m, condition_offsets_list, same_shape=False):
    """Evolve the automaton from the input_row using given rule offsets."""
    print(f"condition offsets list: {condition_offsets_list}")
    
    Z = np.zeros((H - m + 1, L), dtype=int)
    Z = np.append(Z, input_row, axis=0)  # append input row at the bottom
    for i in range(H - m, -1, -1):  # evolve upward
        condition_offsets = condition_offsets_list[H - m - i] if same_shape == False else condition_offsets_list[0]  # get the condition offsets for this row
        for j in range(L):
            neighbor_sum = 0
            for dx, dy in condition_offsets:
                neighbor_sum += Z[i + dx, (j + dy) % L]
            Z[i, j] = 1 if neighbor_sum % 2 == 1 else 0  # parity check

    return Z

def precompute_basis_outputs(H, L, m, condition_indices, same_shape=False):
    """Precompute Z for each basis input (single 1 at position i)."""
    Z_basis = []

    for i in range(m-1):
        input_i = np.zeros((1, (m-1) * L), dtype=int)
        input_i = input_i.reshape(m - 1, L)
        input_i[i, 0] = 1
        Z_first = fill_Z_with_stabilizer_shape(input_i, H, L, m, condition_indices, same_shape=same_shape)
        Z_basis.append(Z_first)

        for j in range(1, L):
            Z_j = np.roll(Z_first, j, axis=1)
            Z_basis.append(Z_j)

    """Print the precomputed Z_basis for debugging"""
    print(f"Precomputed Z_basis for H={H}, L={L}, m={m}:")
    for idx, z in enumerate(Z_basis):
        print(f"Z_basis[{idx}] (index {idx}):\n{z}")

    return Z_basis

def evolve_via_basis(input_row, Z_basis, L, min_weight):
    """Use linearity to compute Z for arbitrary input row."""
    result = np.zeros_like(Z_basis[0])
    indices = np.transpose(np.nonzero(input_row))
    for x, y in indices:
        idx = x * L + y  # compute index in Z_basis
        result = np.bitwise_xor(result, Z_basis[idx])
    return result

from itertools import product

def find_distance_via_basis(Z_basis, H, L, m, pattern_inputs=None):
    """Find the minimum weight Z by iterating over all possible input rows."""
    
    n = H * L
    k = (m - 1) * L
    min_weight = H * L  # initialize large
    kd_over_n = 0

    if pattern_inputs is None:
        iterator = product([0, 1], repeat=k)
    else:
        iterator = pattern_inputs


    for item in iterator:
        if pattern_inputs is None:
            bits = item
            input_row = np.array(bits, dtype=int).reshape(m-1, L)
        else:
            input_row = np.array(item)

        Z = evolve_via_basis(input_row, Z_basis, L, min_weight)
        weight = np.sum(Z)
        if weight > 0:
            if weight < min_weight:
                # New lower weight found: reset lists
                min_weight = weight
                kd_over_n = (k * min_weight) / n
                min_weight_inputs = [input_row.copy()]
                min_weight_outputs = [Z.copy()]
                print(f"Found Z with d={weight}: \n{Z}")
            elif weight == min_weight:
                # Same as current min_weight: add to list
                min_weight_inputs.append(input_row.copy())
                min_weight_outputs.append(Z.copy())
                print(f"Found Z with d={weight}: \n{Z}")
    print(f"best kd/n = {kd_over_n:.2f}")
    return min_weight_inputs, min_weight_outputs, k, min_weight, n, kd_over_n



In [28]:
from itertools import combinations

def rotate_left(x, bits):
    """Rotate bits in integer x left by 1 (with wrap-around)."""
    return ((x << 1) & ((1 << bits) - 1)) | (x >> (bits - 1))

def generate_unique_cyclic_inputs_rowwise_bitwise(m, L, input_weight_limit=None):
    """Optimized row-wise cyclic input generator using bitwise operations."""
    seen = set()
    unique_inputs = []
    k = (m - 1) * L
    if input_weight_limit is None:
        input_weight_limit = k  # You can adjust this limit as needed
    else:
        input_weight_limit = min(input_weight_limit, k)

    print(f"Generating unique inputs with weight up to {input_weight_limit}")

    def reshape_to_row_ints(bits):
        """Convert flat bit list to tuple of integers, one per row."""
        return tuple(int(''.join(str(b) for b in bits[i * L:(i + 1) * L]), 2) for i in range(m - 1))

    # Manually add 1-weight inputs
    for i in range(m - 1):
        row_ints = [0] * (m - 1)
        row_ints[i] = 1 << (L - 1)
        unique_inputs.append(row_ints.copy())
        seen.add(tuple(row_ints))

    print(f"Generated {len(unique_inputs)} unique inputs with weight 1")

    for weight in range(2, input_weight_limit + 1):
        for positions in combinations(range(k), weight):
            flat_bits = [0] * k
            for pos in positions:
                flat_bits[pos] = 1
            row_ints = reshape_to_row_ints(flat_bits)

            # Generate all row-wise cyclic shifts
            rotations = set()
            for r in range(L):
                rotated = tuple(rotate_left(row, L) for row in row_ints)
                if rotated in seen:
                    break
                rotations.add(rotated)
                row_ints = rotated  # Rotate in-place for next r
            else:
                seen.update(rotations)
                unique_inputs.append(list(row_ints))

        print(f"Generated {len(unique_inputs)} unique inputs with weight {weight}")

    # Optionally convert back to 2D arrays
    result = []
    for row_ints in unique_inputs:
        arr = np.array([[int(b) for b in format(val, f'0{L}b')] for val in row_ints], dtype=int)
        result.append(arr)

    return result


In [32]:
H, L = 8, 17
m = 2
stabilizer_shapes = [
    np.array([[0, 0, 1, 0, 0],
              [1, 1, 1, 1, 1],
              [0, 0, 0, 0, 0]])
]

print("Stabilizer shapes:")
print(stabilizer_shapes)
condition_indices_list = []

for shape in stabilizer_shapes:
    condition_indices_list.append(get_condition_indices(shape))

print(f"Condition indices extracted: {condition_indices_list}")

# same stabilizer shape for all rows


Z_basis = precompute_basis_outputs(H, L, m, condition_indices_list, same_shape=True)

print("\nZ_basis precomputed done.")

unique_pattern_inputs = generate_unique_cyclic_inputs_rowwise_bitwise(m, L)

# Find minimum weight Z with Brute Force search
# min_weight_inputs, min_weight_outputs, k, d, n, kd_over_n = find_distance_via_basis(Z_basis, H, L, m)
# print(f"k: {k}, d: {d}, n: {n}, kd/n: {kd_over_n:.2f}")
# Find minimum weight Z with Brute Force search using unique cyclic inputs
print("\nFinding minimum weight Z using unique cyclic inputs...")
print(f"\nlength of whole inputs: {2**((m-1)*L)}")
print(f"\nlength of unique pattern inputs: {len(unique_pattern_inputs)}")
print("Unique cyclic inputs:")

# for input in unique_pattern_inputs:
#     print(input)

min_weight_inputs, min_weight_outputs, k, d, n, kd_over_n = find_distance_via_basis(
    Z_basis, H, L, m, pattern_inputs=unique_pattern_inputs
)
print(f"k: {k}, d: {d}, n: {n}, kd/n: {kd_over_n:.2f}")

Stabilizer shapes:
[array([[0, 0, 1, 0, 0],
       [1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0]])]
Stabilizer shape:
[[0 0 1 0 0]
 [1 1 1 1 1]
 [0 0 0 0 0]]
Rows: 3, Cols: 5
Relative positions:
[[(0, 0), (0, 0), (0, 0), (0, 0), (0, 0)], [(1, -2), (1, -1), (1, 0), (1, 1), (1, 2)], [(2, -2), (2, -1), (2, 0), (2, 1), (2, 2)]]
Condition indices extracted: [[(1, -2), (1, -1), (1, 0), (1, 1), (1, 2)]]
condition offsets list: [[(1, -2), (1, -1), (1, 0), (1, 1), (1, 2)]]
Precomputed Z_basis for H=8, L=17, m=2:
Z_basis[0] (index 0):
[[1 1 0 0 0 1 1 0 0 0 0 1 1 0 0 0 1]
 [1 0 0 0 1 1 0 1 0 0 1 0 1 1 0 0 0]
 [1 1 0 1 1 1 0 0 0 0 0 0 1 1 1 0 1]
 [1 0 0 0 1 0 0 0 1 1 0 0 0 1 0 0 0]
 [1 0 1 0 0 1 1 0 0 0 0 1 1 0 0 1 0]
 [1 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0]
 [1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
Z_basis[1] (index 1):
[[1 1 1 0 0 0 1 1 0 0 0 0 1 1 0 0 0]
 [0 1 0 0 0 1 1 0 1 0 0 1 0 1 1 0 0]
 [1 1 1 0 1 1 1 0 0 0 0 0 0 1 1 1 0]
 [0 1 0 0 0 1 0 0 0 1 1 0 0 0 1 0 0]
 [0 1 0 